In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
# Use the first partition of texas_charges with the fit transformer
attrs = catalog.load('texas_charges.trn.table')['idx'].get_attributes()
wrk = catalog.load('texas_charges.wrk.idx_table').sample()
ref = catalog.load('texas_charges.ref.idx_table').sample()

import pandas as pd
import numpy as np
table = pd.concat([wrk, ref])
table.shape

(11557126, 12)


In [ ]:
Ns = [
    20_000,
    500_000,
    1_000_000,
    2_000_000,
    5_000_000,
    10_000_000
]

N_dom = 1_000_000

In [ ]:
def stub_columns(table, cols):
    table_out = table
    l = len(table)
    for col, dom in cols.items():
        table_out = table_out.assign(**{col: np.random.randint(dom, size=(l, ))})
    return table_out

table_stub = stub_columns(table, {'revenue_code': 357, 'hcpcs_procedure_code': 3018, "modifier_1": 48, "modifier_2": 14, "hcpcs_qualifier": 6})

In [ ]:
from pasteur.hierarchy import rebalance_attributes
from pasteur.marginal import MarginalOracle
from pasteur.utils import LazyDataset, LazyPartition

m = MarginalOracle(attrs, LazyDataset(LazyPartition(lambda: table_stub, lambda: table.shape)))
counts = m.get_counts()
m.close()

new_attrs = rebalance_attributes(counts, attrs)

WARNING  Launching a process pool implicitly. Use `init_pool()` to explicitly control pool creation.                    progress.py:227
WARNING  Rebalancing columns without using Differential Privacy (e_p=inf)                                              hierarchy.py:440


In [ ]:
for name, attr in new_attrs.items():
    for val_name, val in attr.vals.items():
        print(f"{val_name:>25s} | {', '.join(f'{h}:{val.get_domain(h):3d}' for h in range(val.height))}")


             revenue_code | 0:357, 1:280, 2:216, 3:166, 4:128, 5: 98, 6: 76, 7: 58, 8: 45, 9: 35, 10: 27, 11: 21, 12: 16, 13: 12, 14:  8, 15:  5, 16:  4, 17:  3
          hcpcs_qualifier | 0:  6, 1:  5, 2:  4, 3:  3
     hcpcs_procedure_code | 0:3018, 1:2965, 2:2281, 3:1755, 4:1350, 5:1039, 6:799, 7:615, 8:473, 9:364, 10:280, 11:216, 12:166, 13:128, 14: 98, 15: 76, 16: 58, 17: 45, 18: 35, 19: 27, 20: 21, 21: 16, 22: 12, 23:  8, 24:  5, 25:  4, 26:  3
               modifier_1 | 0: 48, 1: 45, 2: 35, 3: 27, 4: 21, 5: 16, 6: 12, 7:  8, 8:  5, 9:  4, 10:  3
               modifier_2 | 0: 14, 1: 12, 2:  8, 3:  5, 4:  4, 5:  3
               modifier_3 | 0:  3, 1:  3
               modifier_4 | 0:  3, 1:  3
    unit_measurement_code | 0:  6, 1:  5, 2:  4, 3:  3
         units_of_service | 0: 21, 1: 21, 2: 16, 3: 12, 4:  8, 5:  5, 6:  4, 7:  2
                unit_rate | 0: 21, 1: 21, 2: 16, 3: 12, 4:  8, 5:  5, 6:  4, 7:  2
          chrgs_line_item | 0: 20, 1: 16, 2: 12, 3:  8, 4:  5, 5:  4

In [ ]:
m2_8 = {'revenue_code': 17, 'hcpcs_qualifier': 3, 'hcpcs_procedure_code': 26, 'modifier_1': 10, 'modifier_2': 5}
m2_12 = {'revenue_code': 17, 'hcpcs_qualifier': 3, 'hcpcs_procedure_code': 26, 'modifier_1': 10, 'modifier_2': 5, 'unit_measurement_code': 2, 'units_of_service': 6}
m2_16 = {'revenue_code': 17, 'hcpcs_qualifier': 3, 'hcpcs_procedure_code': 26, 'modifier_1': 10, 'modifier_2': 5, 'unit_measurement_code': 2, 'units_of_service': 6, 'unit_rate': 6, 'chrgs_line_item': 5}
m2_20 = {'revenue_code': 16, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 23, 'modifier_1': 10, 'modifier_2': 5, 'unit_measurement_code': 2, 'units_of_service': 6, 'unit_rate': 6, 'chrgs_line_item': 5, 'chrgs_non_cov': 5}
m2_24 = {'revenue_code': 16, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 20, 'modifier_1': 8, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 5, 'unit_rate': 5, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}
m2_28 = {'revenue_code': 12, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 18, 'modifier_1': 6, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 5, 'unit_rate': 5, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}
m2_32 = {'revenue_code': 9, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 16, 'modifier_1': 4, 'modifier_2': 5, 'unit_measurement_code': 1, 'units_of_service': 4, 'unit_rate': 4, 'chrgs_line_item': 4, 'chrgs_non_cov': 4}

typ_1 = {'revenue_code': 17, 'hcpcs_qualifier': 3, 'hcpcs_procedure_code': 26, 'modifier_1': 10, 'modifier_2': 5, 'unit_measurement_code': 2, 'units_of_service': 6, 'unit_rate': 6, 'chrgs_line_item': 5}
tst = typ_1

def get_dom(cols):
    dom = 1
    for n, h in cols.items():
        dom *= new_attrs[n][n].get_domain(h)
    return dom

print(f'{get_dom(tst):,}')

62,208


In [ ]:
from pasteur.marginal import AttrSelector, AttrSelectors, expand_table
from pasteur.marginal.native_py import calc_marginal_1way as calc_marginal_native
from pasteur.marginal.numpy import calc_marginal_1way as calc_marginal_numpy

import numpy as np

In [ ]:
import jax
import jax.numpy as jnp

def marginal_jax(ops, dom):
    out = None
    for mult, op in ops:
        if out == None:
            out = mult * op
        else:
            out += mult * op
    return jnp.zeros(dom).at[out].add(1)

marginal_jax = jax.jit(marginal_jax, static_argnums=[1])

def calc_marginal_jax(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelectors,
    out: np.ndarray | None = None
):
    """Calculates the 1 way marginal of the subsections of attributes x"""

    ops = []
    mul = 1
    for attr in reversed(x.values()):
        common = attr.common
        l_mul = 1
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                ops.append((l_mul * mul, jnp.asarray(cols[n][h])))
            else:
                ops.append((l_mul * mul, jnp.asarray(cols_noncommon[n][h])))
            l_mul *= domains[n][h] - common
        mul *= l_mul + common

    marginal_jax(ops, mul).block_until_ready()
    %timeit marginal_jax(ops, mul).block_until_ready()

2000-01-01 00:00:00,000394: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000408: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000250: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [ ]:
def compare_marginals(M, N, desc):
    print("##############")
    print(desc)
    x = {n: AttrSelector(n, 0, {n: h}) for n,h in M.items()}
    cols, cols_noncommon, domains = expand_table(new_attrs, table_stub[:N])

    out16 = np.zeros((get_dom(M),), dtype=np.uint32)
    dom32 = max(get_dom(M), 2**17)
    out32 = np.zeros((dom32,), dtype=np.uint32)

    # vc_df = pd.DataFrame({n: cols[n][h] for n, h in M.items()})
    # %timeit vc_df.value_counts()

    # hdd_np = vc_df.to_numpy()
    # %timeit np.histogramdd(hdd_np, [domains[n][h] for n, h in M.items()])

    # %timeit calc_marginal_numpy(cols, cols_noncommon, domains, x)

    with jax.default_device(jax.devices("cpu")[0]):
        calc_marginal_jax(cols, cols_noncommon, domains, x)
    # # calc_marginal_jax(cols, cols_noncommon, domains, x)

    # %timeit calc_marginal_native(cols, cols_noncommon, domains, x, simd=False, out=out16)
    # %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out32)
    # %timeit calc_marginal_native(cols, cols_noncommon, domains, x, out=out16)

In [ ]:
M = typ_1

for N in Ns:
    compare_marginals(M, N, f"N = {N:,}")

##############
N = 20,000
INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:        xla_bridge.py:355
INFO     Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm".        xla_bridge.py:355
         Available platform names are: Host CUDA Interpreter                                                                           
INFO     Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'          xla_bridge.py:355
INFO     Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client.       xla_bridge.py:355
         Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to                        
         false) to enable this.                                                                                                        
214 µs ± 1.56 µs per l

In [ ]:
Ms = {
    "2^8": m2_8,
    "2^12": m2_12,
    "2^16": m2_16,
    "2^20": m2_20,
    "2^24": m2_24,
    "2^28": m2_28,
    # "2^32": m2_32
}

In [ ]:
for name, M in Ms.items():
    compare_marginals(M, N_dom, f"dom = {name}")

##############
dom = 2^8
8.02 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
##############
dom = 2^12
8.09 ms ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
##############
dom = 2^16
7.92 ms ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
##############
dom = 2^20
8.37 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
##############
dom = 2^24
23.2 ms ± 664 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
##############
dom = 2^28
201 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
